In [1]:
import requests
from bs4 import BeautifulSoup
import string
import pandas as pd

In [2]:
big=string.ascii_uppercase
baseof_letter_url='https://poezja.org/wz/letter/'
dict_of_artists={}
for letter in big:
    letter_url=baseof_letter_url+letter+'/' #link to page with artists for each letter
    main = requests.get(letter_url)
    to_get = BeautifulSoup(main.text)
    for artists in to_get.find_all(class_="col-8 col-sm-12 col-lg-8 author--content"):  #loop for each artists on a page
        dict_of_artists[artists.a.text]=artists.a.get('href')

In [3]:
dict_of_poems={}
for key,values in dict_of_artists.items(): #loop for each arists in dictionary
    profile = requests.get(values)
    content = BeautifulSoup(profile.text)
    sth=content.find_all(id='author-works') 
    for intere in sth:
        for ints in intere.find_all(class_="col-12 col-sm-6 col-md-4 col-lg-3 justify-content-between card"): 
            dict_of_poems[ints.a.get('href')]=[ints.a.text, key]    #getting links and titles for each artist poems

In [4]:
titles_to_df=[]
artists_to_df=[]
poems_to_df=[]
for  key,values in dict_of_poems.items():
    text=[]
    between = requests.get(key)
    book = BeautifulSoup(between.text)
    for x in book.find_all(class_="col-12 col-lg-8"): 
        text.append(x.text)
    try:
        titles_to_df.append(text[0])
        artists_to_df.append(text[1])
        poems_to_df.append(text[2])
    except IndexError:
        pass

In [5]:
df=pd.DataFrame(
    {"title" : titles_to_df,
     "poem" : poems_to_df,
     "artist" : artists_to_df
    })
df.head()

,title,poem,artist
0,\nNoc taka piękna!\n,\n\nNoc taka piękna!... Z błękitów patrzy gwia...,\n\n\nAutor:\nBogusław Adamowicz\n\n\n
1,\nNagość\n,"\n\nWszystko co kocha, obsłon pragnie zdzierać...",\n\n\nAutor:\nBogusław Adamowicz\n\n\n
2,\nWstyd\n,\n\nNad łożem Niewinności wiotkiej i dziewicze...,\n\n\nAutor:\nBogusław Adamowicz\n\n\n
3,\nZa kim?\n,"\n\nZdaje się, że schodzę w jakąś otchłań ciem...",\n\n\nAutor:\nBogusław Adamowicz\n\n\n
4,\nDwa jakieś kwiaty...\n,\n\nDwa jakieś kwiaty na nieznanej łące\nZnęci...,\n\n\nAutor:\nBogusław Adamowicz\n\n\n


In [6]:
df['title']=df['title'].str.replace('\n','')
df['title']=df['title'].str.strip()
df['artist']=df['artist'].str.replace('Autor:','')
df['artist']=df['artist'].str.replace('\n',' ')
df['artist']=df['artist'].str.strip()
df.head()

,title,poem,artist
0,Noc taka piękna!,\n\nNoc taka piękna!... Z błękitów patrzy gwia...,Bogusław Adamowicz
1,Nagość,"\n\nWszystko co kocha, obsłon pragnie zdzierać...",Bogusław Adamowicz
2,Wstyd,\n\nNad łożem Niewinności wiotkiej i dziewicze...,Bogusław Adamowicz
3,Za kim?,"\n\nZdaje się, że schodzę w jakąś otchłań ciem...",Bogusław Adamowicz
4,Dwa jakieś kwiaty...,\n\nDwa jakieś kwiaty na nieznanej łące\nZnęci...,Bogusław Adamowicz


In [7]:
df.to_csv('poems.csv', index=False)